In [3]:
import pandas_datareader.data as web
import datetime
import pandas as pd

# =======================
# 1. Configuration
# =======================
# Time
start_date = datetime.datetime(2000, 1, 1)
end_date   = datetime.date.today()

# Dictionary containing FRED codes to fetch
# Format: 'Desired column name': 'FRED code'
fred_series = {
    # Inflation
    'UNRATE': 'UNRATE',
    'FEDFUNDS': 'FEDFUNDS',
    'CPI': 'CPIAUCSL',
    'CPI_CORE': 'CPILFESL',
    'PCEPI': 'PCEPI',
    'PCE_CORE': 'PCEPILFE',
    'T5YIE': 'T5YIE',
    # Growth & Production
    'INDPRO': 'INDPRO',
    'RSALES': 'RSALES',
    # Labor Market
    'PAYEMS': 'PAYEMS',
    'AHE': 'AHETPI',        # Updated code for Average Hourly Earnings
    'CIVPART': 'CIVPART',
    # Monetary Policy & Financial Conditions
    'EFFR': 'EFFR',
    'FED_ASSETS': 'WALCL',
    # Sentiment & Confidence
    'UMCSENT': 'UMCSENT',
    # Credit Spread
    'BAA_YIELD': 'BAA',
    # Bond Yields
    'DGS1MO': 'DGS1MO',
    'DGS3MO': 'DGS3MO',
    'DGS6MO': 'DGS6MO',
    'DGS1': 'DGS1',
    'DGS2': 'DGS2',
    'DGS3': 'DGS3',
    'DGS5': 'DGS5',
    'DGS7': 'DGS7',
    'DGS10': 'DGS10',
    'DGS20': 'DGS20',
    'DGS30': 'DGS30',
}

# =======================
# 2. Data retrieval loop (More robust)
# =======================
all_data = []
print("Starting data retrieval from FRED...")

for name, code in fred_series.items():
    try:
        series = web.DataReader(code, "fred", start_date, end_date)
        series.columns = [name]
        all_data.append(series)
        print(f"  [SUCCESS] Retrieved data for: {name} ({code})")
    except Exception as e:
        # If there's an error, print the problematic code and continue
        print(f"  [ERROR] Could not retrieve data for {name} ({code}). Error: {e}")

# =======================
# 3. Combine and process data
# =======================
if not all_data:
    print("\nNo data was downloaded. Stopping program.")
else:
    print("\nData retrieval complete. Starting combination and processing...")
    # Combine all retrieved DataFrames
    df = pd.concat(all_data, axis=1)

    # Process data
    df = df.ffill() # Fill-forward to synchronize frequency
    df = df.dropna() # Remove rows with NaN values

    # =======================
    # 4. Feature Engineering
    # =======================
    # Check if required columns exist before creating new features
    if "DGS10" in df.columns and "DGS2" in df.columns:
        df["YIELD_CURVE_10Y_2Y"] = df["DGS10"] - df["DGS2"]
    if "DGS10" in df.columns and "DGS3MO" in df.columns:
        df["YIELD_CURVE_10Y_3M"] = df["DGS10"] - df["DGS3MO"]

    if "BAA_YIELD" in df.columns and "DGS10" in df.columns:
        df["BAA_SPREAD"] = df["BAA_YIELD"] - df["DGS10"]

    if "CPI" in df.columns:
        df["CPI_YOY_CHANGE"] = df["CPI"].pct_change(periods=12) * 100
    if "PAYEMS" in df.columns:
        df["PAYEMS_MOM_CHANGE"] = df["PAYEMS"].diff()
        
    # Remove NaN values generated after calculations (e.g., first 12 months of CPI_YOY_CHANGE)
    df = df.dropna()

    # =======================
    # 5. Export results
    # =======================
    output_file = "macro_treasury_full_expanded.csv"
    df.to_csv(output_file, index=True)

    print(f"\n[COMPLETE] Data has been saved to file: {output_file}")
    print("\nFirst 5 rows of the final DataFrame:")
    print(df.head())
    print("\nLast 5 rows of the final DataFrame:")
    print(df.tail())
    print("\nDataFrame information:")
    df.info()

Starting data retrieval from FRED...
  [SUCCESS] Retrieved data for: UNRATE (UNRATE)
  [SUCCESS] Retrieved data for: FEDFUNDS (FEDFUNDS)
  [SUCCESS] Retrieved data for: CPI (CPIAUCSL)
  [SUCCESS] Retrieved data for: CPI_CORE (CPILFESL)
  [SUCCESS] Retrieved data for: PCEPI (PCEPI)
  [SUCCESS] Retrieved data for: PCE_CORE (PCEPILFE)
  [SUCCESS] Retrieved data for: T5YIE (T5YIE)
  [SUCCESS] Retrieved data for: INDPRO (INDPRO)
  [SUCCESS] Retrieved data for: RSALES (RSALES)
  [SUCCESS] Retrieved data for: PAYEMS (PAYEMS)
  [SUCCESS] Retrieved data for: AHE (AHETPI)
  [SUCCESS] Retrieved data for: CIVPART (CIVPART)
  [SUCCESS] Retrieved data for: EFFR (EFFR)
  [SUCCESS] Retrieved data for: FED_ASSETS (WALCL)
  [SUCCESS] Retrieved data for: UMCSENT (UMCSENT)
  [SUCCESS] Retrieved data for: BAA_YIELD (BAA)
  [SUCCESS] Retrieved data for: DGS1MO (DGS1MO)
  [SUCCESS] Retrieved data for: DGS3MO (DGS3MO)
  [SUCCESS] Retrieved data for: DGS6MO (DGS6MO)
  [SUCCESS] Retrieved data for: DGS1 (DGS1)
